In [69]:
!pip install pyinterval

In [70]:
from interval import interval, imath
import pandas as pd
import math

In [71]:
def calculate_width(interv):
    return sum([x.sup - x.inf for x in interv])

def calculate_middle_point(interv):
    return (interv[0].sup + interv[0].inf) / 2

def to_string(interv):
    return f"[{interv[0].inf:4.3f}, {interv[0].sup:4.3f}]"

In [72]:
def calculate_volterra1(f, expected, k, y0, a, b, c, d, p):
    h = (b - a) / p
    x = []
    y = []
    exp = []
    exp.append(interval[expected(a),expected(a)])
    for i in range(0, p):
        xi = a + i*h
        xinext = xi + h
        x.append(interval[xi, xinext])
        sum = 0
        for j in range(0, i):
            sum += k(x[i], x[j], y0[j])
        y.append(f(x[i]) + h*sum + interval[0,h]*k(x[i], x[i], y0[i]))
        exp.append(expected(xi))
    u = []
    u.append(y0[0])
    for i in range(0,p-1):
        u.append(y[i]&y[i+1])
    return x, y, u, exp

In [73]:
a = 0
b = math.sqrt(3)-1
c = 0
d = 2
p = 4
n = 5
y0 = []
for i in range(p):
    y0.append(interval[c,d])
func = lambda x: x**2
k = lambda x,s,y : (-s)*y
expected = lambda x: 2-2*math.exp(-x**2/2)

In [74]:
for iteration in range(n):
    print(f"\nn={iteration+1}")
    x, y, u, exp = calculate_volterra1(func, expected, k, y0, a, b, c, d, p)
    tbl = pd.DataFrame(columns=['xi', 'yi', 'ui', 'm(ui)', 'expected', 'e'])
    for i in range(p):
        mid_point=calculate_middle_point(u[i])
        real = expected(x[i][0].inf)
        tbl.loc[len(tbl)] = [to_string(x[i]),to_string(y[i]),to_string(u[i]),f"{mid_point:4.4}",f"{real:4.4}",f"{abs(mid_point-real):4.4}"]
    y0 = y
    print(tbl)


n=1
               xi               yi               ui     m(ui) expected  \
0  [0.000, 0.183]  [-0.067, 0.033]   [0.000, 2.000]       1.0      0.0   
1  [0.183, 0.366]  [-0.167, 0.134]  [-0.067, 0.033]  -0.01675  0.03321   
2  [0.366, 0.549]  [-0.268, 0.301]  [-0.167, 0.134]  -0.01675   0.1296   
3  [0.549, 0.732]  [-0.368, 0.536]  [-0.268, 0.301]   0.01675   0.2798   

         e  
0      1.0  
1  0.04996  
2   0.1463  
3   0.2631  

n=2
               xi               yi               ui     m(ui) expected  \
0  [0.000, 0.183]  [-0.001, 0.036]  [-0.067, 0.033]  -0.01675      0.0   
1  [0.183, 0.366]   [0.023, 0.147]   [0.023, 0.036]   0.02957  0.03321   
2  [0.366, 0.549]   [0.094, 0.342]   [0.094, 0.147]    0.1205   0.1296   
3  [0.549, 0.732]   [0.189, 0.626]   [0.189, 0.342]    0.2655   0.2798   

          e  
0   0.01675  
1  0.003647  
2  0.009073  
3   0.01428  

n=3
               xi               yi               ui    m(ui) expected  \
0  [0.000, 0.183]  [-0.001, 0.034] 